In [1]:
import pandas as pd

from jesse import research, helpers

from custom_indicators.all_features import feature_bundle

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

_, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

_, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

_, trading_1h = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1h",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [2]:
features_1m = {
    f"1m_{k}": v for k, v in feature_bundle(trading_1m, sequential=True).items()
}
df_1m = pd.DataFrame(
    features_1m,
    index=pd.DatetimeIndex([helpers.timestamp_to_time(t) for t in trading_1m[:, 0]]),
)
df_1m = df_1m.resample("3min").agg({col: "last" for col in df_1m.columns})

features_3m = {
    f"3m_{k}": v for k, v in feature_bundle(trading_3m, sequential=True).items()
}
df_3m = pd.DataFrame(
    features_3m,
    index=pd.DatetimeIndex([helpers.timestamp_to_time(t) for t in trading_3m[:, 0]]),
)

features_15m = {
    f"15m_{k}": v for k, v in feature_bundle(trading_15m, sequential=True).items()
}
df_15m = pd.DataFrame(
    features_15m,
    index=pd.DatetimeIndex([helpers.timestamp_to_time(t) for t in trading_15m[:, 0]]),
)
df_15m = df_15m.resample("3min").agg({col: "last" for col in df_15m.columns}).ffill()

features_1h = {
    f"1h_{k}": v for k, v in feature_bundle(trading_1h, sequential=True).items()
}
df_1h = pd.DataFrame(
    features_1h,
    index=pd.DatetimeIndex([helpers.timestamp_to_time(t) for t in trading_1h[:, 0]]),
)
df_1h = df_1h.resample("3min").agg({col: "last" for col in df_1h.columns}).ffill()

df_features = pd.concat([df_1m, df_3m, df_15m, df_1h], axis=1).ffill()
print(df_features.shape)
df_features.tail()

NameError: name 'feature_bundle' is not defined

Bad pipe message: %s [b'\r\x03\x96\xb6\x7f4\xc1\xb2\x7f\xeah\'\x00\xce?\xceN\xf8 TB*\xeb\xaeG\xf8\x9b\x95w1v\xbb"X\xc7\xb3*y\xa4-\xf7\x97\x08^\x9a\xaf\x8c\x17\x84\x86=\x00*ZZ\x13\x01\x13\x02\x13\x03\xc0,\xc0+\xcc\xa9\xc00\xc0/\xcc\xa8\xc0\n\xc0\t\xc0\x14\xc0\x13\x00\x9d\x00\x9c\x005\x00/\xc0\x08\xc0\x12\x00\n\x01\x00\x01\x89\xca\xca\x00\x00\x00\x00\x00\x0e\x00\x0c\x00\x00\tlocalhost\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\x0c\x00\njj\x00\x1d\x00\x17\x00\x18\x00\x19\x00\x0b\x00\x02\x01\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\r\x00']
Bad pipe message: %s [b'\x14\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x08\x05\x05\x01\x08\x06\x06\x01\x02\x01']
Bad pipe message: %s [b'\x13\xfbG4\xa2\x9c:.\x80n\\\x95\x06\xe4-\xdb\xd6\x10 \xeaZ\xba,\xef?n\xde\x1b\\LS\xb6\xbe\xb8$\xb5\xadN\x9c|\xae$\x0c\x11N@b\xd0\xbb\xd2\x9f\x00,\xea\xea\x13\x01\x13\x02\x13\x03\xc0,\xc0+\xcc\xa9\xc00\xc0/\xcc\xa8\xc0\n\xc0\t\xc0\x14\xc0\x13\x00\x9d\x00\x9c\x005\x00/\xc0\x08

In [3]:
df_features.isna().sum(axis=0).sort_values(ascending=False)

1h_dft_dom_cycle_lag3    1000
1h_dft_dom_cycle_ddt      980
1h_dft_dom_cycle_lag2     980
1h_acp_pwr_24             960
1h_acp_pwr_13             960
                         ... 
3m_conv_35                  0
3m_conv_36                  0
3m_conv_37                  0
3m_conv_38                  0
15m_ac_0                    0
Length: 1684, dtype: int64

In [4]:
df_features.to_parquet("data/features_3m.parquet")